# 🚀 GenAI Environment Setup - CoursIA

**Module :** 00-GenAI-Environment  
**Niveau :** 🟢 Débutant  
**Technologies :** OpenRouter, OpenAI, Docker, Configuration  
**Durée estimée :** 15 minutes  

## 🎯 Objectifs d'Apprentissage

- [ ] Configurer l'environnement GenAI CoursIA complet
- [ ] Valider les APIs OpenRouter et OpenAI
- [ ] Tester la connectivité des services Docker (optionnel)

## 📚 Prérequis

- Fichier .env configuré avec les clés API
- Python 3.11+ avec dépendances installées
- Accès réseau pour tests API


In [ ]:
# Paramètres Papermill - JAMAIS modifier ce commentaire
# Parameters cell for MCP compatibility

# Configuration pour exécution batch/MCP
notebook_mode = "interactive"  # "interactive" ou "batch" 
api_provider = "openrouter"    # "openrouter", "openai", "local"
skip_widgets = False           # True pour mode batch MCP
debug_level = "INFO"           # "DEBUG", "INFO", "WARNING", "ERROR"
test_generation = True         # Effectuer un test de génération
validation_mode = "complete"   # "quick", "standard", "complete"

In [ ]:
# 🎯 OBJECTIF : Setup environnement GenAI standardisé CoursIA
# 📝 EXPLICATION : Configuration et validation complète des APIs et services
# ⚡ ACTION : Import des dépendances et chargement configuration

# Setup standardisé CoursIA GenAI
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime
import json
import requests
import asyncio
from typing import Dict, List, Optional, Any

# Configuration paths CoursIA
GENAI_ROOT = Path.cwd() if '__file__' not in locals() else Path(__file__).parent
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

ENV_FILE = GENAI_ROOT / '.env'
SHARED_PATH = GENAI_ROOT / 'shared' / 'helpers'

# Chargement environnement
config_loaded = False
if ENV_FILE.exists():
    load_dotenv(ENV_FILE)
    config_loaded = True
    print(f"✅ Configuration chargée depuis {ENV_FILE}")
else:
    print(f"⚠️  Fichier .env non trouvé : {ENV_FILE}")
    print("📖 Copiez .env.template vers .env et configurez vos clés API")

# Ajout helpers partagés
if SHARED_PATH.exists():
    sys.path.insert(0, str(SHARED_PATH.parent))
    print(f"📁 Helpers partagés ajoutés: {SHARED_PATH}")

print(f"🎨 GenAI CoursIA - Environment Setup")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Mode: {notebook_mode}, Provider: {api_provider}")

In [ ]:
# 🔧 PARAMÈTRES : Validation environnement et APIs
# 📊 RÉSULTATS ATTENDUS : Status des APIs et configuration

def validate_genai_environment() -> Dict[str, Any]:
    """Validation environnement GenAI CoursIA"""
    status = {
        "config_loaded": config_loaded,
        "apis": {
            "openrouter": False,
            "openai": False
        },
        "docker_services": {
            "enabled": False,
            "flux": False,
            "sd": False,
            "comfyui": False
        },
        "environment": {
            "output_dir": None,
            "log_level": debug_level,
            "timeout": None
        }
    }
    
    # Validation APIs
    if os.getenv("OPENROUTER_API_KEY"):
        status["apis"]["openrouter"] = True
        print("✅ OpenRouter API configuré")
    else:
        print("❌ OpenRouter API manquant - Variable OPENROUTER_API_KEY")
    
    if os.getenv("OPENAI_API_KEY"):
        status["apis"]["openai"] = True
        print("✅ OpenAI API disponible")
    else:
        print("⚠️  OpenAI API manquant - Variable OPENAI_API_KEY (optionnel)")
    
    # Configuration Docker
    docker_enabled = os.getenv("DOCKER_ENABLED", "false").lower() == "true"
    status["docker_services"]["enabled"] = docker_enabled
    
    if docker_enabled:
        print("🐳 Docker activé - Test des services locaux...")
        # Test services Docker (sans bloquer)
        docker_services = {
            "flux": os.getenv("FLUX_API_URL", "http://localhost:8189"),
            "sd": os.getenv("SD_API_URL", "http://localhost:8190"),
            "comfyui": os.getenv("COMFYUI_API_URL", "http://localhost:8191")
        }
        
        for service, url in docker_services.items():
            try:
                response = requests.get(f"{url}/health", timeout=5)
                if response.status_code == 200:
                    status["docker_services"][service] = True
                    print(f"✅ Service {service}: {url}")
                else:
                    print(f"⚠️  Service {service}: Status {response.status_code}")
            except:
                print(f"❌ Service {service}: Non accessible ({url})")
    else:
        print("📱 Mode cloud uniquement (Docker désactivé)")
    
    # Configuration environnement
    status["environment"]["output_dir"] = os.getenv("GENAI_OUTPUT_DIR", "outputs/generated")
    status["environment"]["timeout"] = int(os.getenv("GENAI_TIMEOUT_SECONDS", "300"))
    
    # Création répertoire de sortie
    output_dir = Path(status["environment"]["output_dir"])
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"📁 Répertoire de sortie: {output_dir}")
    
    return status

# Validation automatique
environment_status = validate_genai_environment()

# Affichage résumé
apis_available = sum(environment_status["apis"].values())
docker_services = sum(environment_status["docker_services"].values()) - (1 if environment_status["docker_services"]["enabled"] else 0)

print(f"\n📊 RÉSUMÉ VALIDATION")
print(f"APIs disponibles: {apis_available}/2")
print(f"Services Docker: {docker_services}/3 ({'activé' if environment_status['docker_services']['enabled'] else 'désactivé'})")
print(f"Configuration: {'✅ Complète' if config_loaded and apis_available > 0 else '⚠️  Partielle'}")

In [ ]:
# 💡 ASTUCE : Test de génération simple pour validation fonctionnelle
# ⚠️  ATTENTION : Ce test utilise des crédits API (minimal)

async def test_image_generation():
    """Test de génération d'image pour validation"""
    
    if not test_generation:
        print("⏭️  Test de génération désactivé (test_generation=False)")
        return None
        
    if not any(environment_status["apis"].values()):
        print("❌ Aucune API disponible pour le test")
        return None
    
    print("🧪 Test génération d'image...")
    
    # Test simple avec OpenRouter (prioritaire)
    if environment_status["apis"]["openrouter"]:
        try:
            headers = {
                'Authorization': f'Bearer {os.getenv("OPENROUTER_API_KEY")}',
                'Content-Type': 'application/json'
            }
            
            # Test avec un modèle simple
            payload = {
                'model': 'openai/dall-e-3',
                'prompt': 'A simple test image: colorful abstract shapes',
                'size': '1024x1024',
                'quality': 'standard',
                'n': 1
            }
            
            print("📡 Appel API OpenRouter...")
            response = requests.post(
                'https://openrouter.ai/api/v1/images/generations',
                headers=headers,
                json=payload,
                timeout=60
            )
            
            if response.status_code == 200:
                result = response.json()
                print("✅ Test génération réussi !")
                print(f"🎨 Modèle: {payload['model']}")
                print(f"📝 Prompt: {payload['prompt']}")
                
                if 'data' in result and len(result['data']) > 0:
                    image_url = result['data'][0].get('url')
                    if image_url:
                        print(f"🖼️  Image générée: {image_url[:50]}...")
                        return {
                            'success': True,
                            'provider': 'openrouter',
                            'model': payload['model'],
                            'image_url': image_url
                        }
                
                return {'success': True, 'provider': 'openrouter'}
            else:
                print(f"⚠️  Erreur API: {response.status_code}")
                print(f"Response: {response.text[:200]}...")
                
        except Exception as e:
            print(f"❌ Erreur test OpenRouter: {str(e)[:100]}...")
    
    print("✅ Validation APIs complétée (génération non testée)")
    return {'success': False, 'reason': 'api_available_but_generation_failed'}

# Exécution test si configuré
if validation_mode in ["standard", "complete"]:
    test_result = await test_image_generation()
else:
    print("⏭️  Test génération sauté (validation_mode='quick')")
    test_result = None

In [ ]:
# 📋 RAPPORT FINAL : Configuration et recommandations

# Génération rapport complet
setup_report = {
    "timestamp": datetime.now().isoformat(),
    "notebook_info": {
        "module": "00-GenAI-Environment",
        "notebook": "00-1-Environment-Setup",
        "mode": notebook_mode,
        "provider": api_provider
    },
    "environment_status": environment_status,
    "test_result": test_result,
    "validation_level": validation_mode,
    "recommendations": []
}

# Génération recommandations
if not config_loaded:
    setup_report["recommendations"].append("Configurer le fichier .env avec les clés API")

if not any(environment_status["apis"].values()):
    setup_report["recommendations"].append("Ajouter au moins une clé API (OpenRouter recommandé)")

if environment_status["docker_services"]["enabled"] and docker_services == 0:
    setup_report["recommendations"].append("Démarrer les services Docker ou désactiver DOCKER_ENABLED")

if test_result and not test_result.get('success'):
    setup_report["recommendations"].append("Vérifier la validité des clés API et les quotas")

# Calcul score de santé
health_score = 0
if config_loaded: health_score += 25
if any(environment_status["apis"].values()): health_score += 50
if docker_services > 0: health_score += 15
if test_result and test_result.get('success'): health_score += 10

setup_report["health_score"] = health_score

# Affichage rapport
print("\n" + "="*50)
print("📋 RAPPORT CONFIGURATION GENAI COURSIA")
print("="*50)

print(f"🏥 Score de santé: {health_score}/100")
print(f"⚙️  Configuration: {'✅ Chargée' if config_loaded else '❌ Manquante'}")
print(f"🌐 APIs disponibles: {apis_available}/2")
print(f"🐳 Services Docker: {docker_services}/3")

if test_result:
    if test_result.get('success'):
        print(f"✅ Test génération: Réussi ({test_result.get('provider', 'N/A')})")
    else:
        print(f"⚠️  Test génération: Échoué")

# Recommandations
if setup_report["recommendations"]:
    print("\n💡 RECOMMANDATIONS:")
    for i, rec in enumerate(setup_report["recommendations"], 1):
        print(f"{i}. {rec}")

# Status final
if health_score >= 75:
    status_final = "🎉 Environnement optimal - Tous les modules GenAI disponibles"
elif health_score >= 50:
    status_final = "✅ Environnement fonctionnel - Quelques optimisations possibles"
elif health_score >= 25:
    status_final = "⚠️  Environnement partiel - Configuration à compléter"
else:
    status_final = "❌ Environnement non configuré - Voir recommandations"

print(f"\n{status_final}")

# Sauvegarde rapport
report_path = Path(environment_status["environment"]["output_dir"]) / f"environment_setup_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
report_path.parent.mkdir(parents=True, exist_ok=True)

with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(setup_report, f, indent=2, ensure_ascii=False, default=str)

print(f"\n💾 Rapport sauvegardé: {report_path}")
print("\n🚀 Environnement GenAI CoursIA prêt !")

# Export pour les autres notebooks
GENAI_ENVIRONMENT_STATUS = environment_status
GENAI_HEALTH_SCORE = health_score